In [1]:
import os
import sys
from googleapiclient import sample_tools
import pymongo
import pandas
from datetime import date, timedelta

In [2]:
# DB
clientDB = pymongo.MongoClient("mongodb://___") # replace
storage = clientDB["___"] # replace
collection = storage["___"] # replace

In [3]:
property_uri = '___' # replace GSC property uri

sdate = date(YYYY,M,D) # replace YYYY,M,D
edate = date(YYYY,M,D) # replace YYYY,M,D
all_dates = pandas.date_range(sdate,edate-timedelta(days=1),freq='d').strftime('%Y-%m-%d').tolist()

In [4]:
# Set os.path.abspath (./) in a way, that client_secrets.json can be found
# https://console.cloud.google.com/ obtain file here for GSC API
service, flags = sample_tools.init([], 'searchconsole', 'v1',  __doc__, os.path.abspath('./code/'),
                                   scope='https://www.googleapis.com/auth/webmasters.readonly')

In [5]:
def getValues(r):
    return [r[0]['clicks'], r[0]['impressions'], r[0]['ctr'], r[0]['position']]

In [6]:
for d in all_dates:
    
    # Total Request
    TOTAL_REQUEST = {
      'startDate': d,
      'endDate': d,
      'searchType': 'WEB',
     'rowLimit': 1
    }
    
    '''
    e.g. German Traffic
    DE_REQUEST = {
      'startDate': d,
      'endDate': d,
      'searchType': 'WEB',
      'dimensionFilterGroups': [{
          'filters': [{
              'dimension': 'country',
              'expression': 'DEU'
          }]
      }],
     'rowLimit': 1
      }
    '''
    
    # Save Request
    TOTAL_RESPONSE = service.searchanalytics().query(siteUrl=property_uri, body=TOTAL_REQUEST).execute()
    rows_TTL = getValues(TOTAL_RESPONSE['rows'])
    
    # Import entry
    temp = {}
    temp["date"] = d
    temp["clicks"] = rows_TTL[0]
    temp["imprs"] = rows_TTL[1]
    temp["ctr"] = rows_TTL[2]
    temp["pos"] = rows_TTL[3]
    collection.insert_one(temp)